In [464]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [465]:
mapdl.clear()
mapdl.prep7()


*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   17:44:21  APR 16, 2023 CP=      0.125

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [466]:
# Define Material Property
E = 210e9
nu = 0.3
YS = 240e6
# Defining Geometry
a = 0.02 # Crack length
c = 0.03 # Thickness
theta1 = np.pi/20
theta2 = np.pi/5
a1 = 0.02
a2 = 0.025
x = 0.005

In [467]:
# Define Material Property

E = 210e9
nu = 0.3
YS = 240e6
a = 0.011
    # Defining Geometry 
    # Corners of the rectangle
m0 = mapdl.k("", 0,-0.1)
m1 = mapdl.k("",0,0.1)
m2 = mapdl.k("",x-0.0025,0.1)
m3 = mapdl.k("",x-0.0025,0.1002)
m4 = mapdl.k("",x+0.0025,0.1002)
m5 =mapdl.k("",x+0.0025,0.1)
m6 = mapdl.k("", 0.1, 0.1)
m7 = mapdl.k("", 0.1, -0.1)
main = mapdl.a(m0,m1,m2,m3,m4,m5,m6,m7)

# Defining points for the crack
b = 0.0001
beta = np.pi/12
s1 = np.tan(theta)
s2 = np.tan(theta+beta)# lower side
s3 = np.tan(theta-beta)# upper side

d = a*(np.sin(theta)-np.cos(theta)*s2)
e = a*(np.sin(theta)-np.cos(theta)*s3)

x1 = (b+d)/(s1-s2)
y1 = s1*x1-b
x3 = (b-e)/(s3-s1)
y3 = s1*x3+b



In [468]:

# Crack geometry
k0 = mapdl.k("",0,-b)
#x1 = 0.8*a*np.cos(theta)
#y1= np.tan(theta)*x1-0.001
k1 = mapdl.k("",x1,y1)
k2 = mapdl.k("",a*np.cos(theta),a*np.sin(theta))
#x3 =  0.8*a*np.cos(theta)
#y3= np.tan(theta)*x1+0.001
k3 = mapdl.k("",x3,y3)
k4 = mapdl.k("",0,b)
crack = mapdl.a(k0,k1,k2,k3,k4)
aout = mapdl.asba(main, crack)
mapdl.vext(aout, dz=0.005)
mapdl.vplot(show_lines=True, quality=5)

Widget(value="<iframe src='http://localhost:50968/index.html?ui=P_0x1ae13e59fc0_85&reconnect=auto' style='widt…

In [469]:
mapdl.et(1,"SOLID285")
    # Define the Mesh
mapdl.smrtsize(1)
mapdl.aesize("ALL", 0.003)
mapdl.vmesh(1)

mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")

Widget(value="<iframe src='http://localhost:50968/index.html?ui=P_0x1aec1c08d30_86&reconnect=auto' style='widt…


mapdl.et(1,"SOLID285")
    # Define the Mesh
mapdl.smrtsize(2)
mapdl.aesize("ALL", 0.003)
mapdl.vmesh(1)
scon1 = mapdl.k(10000,0.1,0,0)
scon2 = mapdl.k(10001,0.2,0,0)
mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")


In [470]:
# Using the SI units
press= 250e6
mapdl.units("SI")  # SI - International system (m, kg, s, K).

# Define a material (nominal steel in SI)
mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio
# Fix the lower side.
mapdl.nsel("S", "LOC", "Y", -0.1)
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")
mapdl.d("ALL", "UX")

mapdl.nsel("S", "LOC", "Y", 0.1)
mapdl.cp(5, "UX", "ALL")

mapdl.asel("S",'LOC',"Y",0.1002)
mapdl.sfa('ALL', '', 'PRES', -press)
mapdl.allsel(mute=True)
mapdl.run("/SOLU")
mapdl.antype("STATIC")
mapdl.solve()
mapdl.finish(mute=True)
mapdl.post1()
mapdl.path('P',3,5)
node1 = mapdl.queries.node(a*np.cos(theta),a*np.sin(theta),0)
x2 = 0.6*a
y2 = s1*x2-b
node2 = mapdl.queries.node(x2,y2,0)
node3 = mapdl.queries.node(0,-b,0)
mapdl.ppath(1, node1)
mapdl.ppath(2, node2)
mapdl.ppath(3, node3)
mapdl.local(11,0,a*np.cos(theta),a*np.sin(theta),0,theta*180/np.pi)


COORDINATE SYSTEM         11 DEFINITION.  TYPE= 0  (CARTESIAN)  
   XC,YC,ZC= 0.10865E-01 0.17208E-02  0.0000    
   ANGLES=    9.00    0.00    0.00   PARAMETERS=   1.000   1.000
   ORIENTATION=  0.99  0.16  0.00  -0.16  0.99  0.00   0.00  0.00  1.00

 ACTIVE COORDINATE SYSTEM SET TO        11  (CARTESIAN)

In [471]:
mapdl.csys(11)
k = mapdl.kcalc(0,1,0,0)
result = mapdl.result


In [472]:
 words = k.split(' ')
    
    # Extracting the K1 value from the String output
for i in range(len(words)):
    if words[i]=='KI':
        x=i
K1 = words[x+3][:-1] 
print(f"K1 is equal to {K1}")
K1

K1 is equal to 0.46882E+08


'0.46882E+08'

In [446]:
mapdl.exit()